<h1><center>ЛОГИРУЕМ МОДЕЛЬ</h1></center>

Пора продолжить работу над проблемой оттока клиентов! После первой итерации вы получили базовое решение — модель логистической регрессии. Вы также провели свой первый эксперимент и залогировали основные артефакты и сам набор данных.
<br><br>Прежде чем перейти к логированию модели в MLflow, важно убедиться, что вам доступна сама модель, а MLflow настроен в соответствии с предыдущими уроками. 
<br><br>
1.(**команда sh run_mlflow_server.sh делает все шаги первого этапа**) Поднимите MLflow, используя предложенный ранее вариант конфигурации:

- локальный Tracking Server на виртуальной машине,
- удалённые хранилище экспериментов и хранилище артефактов.

Не забудьте настроить соответствующие переменные окружения для подключения к базе данных Postgres и объектному хранилищу S3.

2.(**код в этом блокноте**) Определите необходимые глобальные переменные. Одной из них будет EXPERIMENT_NAME, которую вы уже использовали. Вторую переменную, RUN_NAME, вы можете назвать самостоятельно, чтобы ясно отразить суть запуска.


In [ ]:
import os
import mlflow
import joblib
from mlflow.tracking import MlflowClient
import pandas as pd
import random
from sklearn.model_selection import train_test_split

# определяем основные credentials, которые нужны для подключения к MLflow
# важно, что credentials мы передаём для себя как пользователей Tracking Service
# у вас должен быть доступ к бакету, в который вы будете складывать артефакты
os.environ["MLFLOW_S3_ENDPOINT_URL"] = "https://storage.yandexcloud.net" #endpoint бакета от YandexCloud
os.environ["AWS_ACCESS_KEY_ID"] = 'YCAJE3Nlz8iDILW5VTYM1ihQB' # os.getenv("AWS_ACCESS_KEY_ID") # получаем id ключа бакета, к которому подключён MLFlow, из .env
os.environ["AWS_SECRET_ACCESS_KEY"] = 'YCPjvS7uwhvJpUj3bKm8X-IX4QAwBIVsvX61IL44' # os.getenv("AWS_SECRET_ACCESS_KEY") #  получаем ключ бакета, к которому подключён MLFlow, из .env

# определяем глобальные переменные
# поднимаем MLflow локально
TRACKING_SERVER_HOST = "127.0.0.1"
TRACKING_SERVER_PORT = 5000

YOUR_NAME = "unique_experiment-3" # введите своё имя для создания уникального эксперимента
assert YOUR_NAME, "alexnikzotov"

# название тестового эксперимента и запуска (run) внутри него
EXPERIMENT_NAME = f"test_connection_experiment_{YOUR_NAME}"
RUN_NAME = "test_connection_run"

# тестовые данные
METRIC_NAME = "test_metric"
METRIC_VALUE = 0

# устанавливаем host, который будет отслеживать наши эксперименты
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")

# создаём тестовый эксперимент и записываем в него тестовую информацию
experiment_id = mlflow.create_experiment(EXPERIMENT_NAME)
with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    run_id = run.info.run_id
    
    mlflow.log_metric(METRIC_NAME, METRIC_VALUE) 

3.Для логирования используйте либо уже имеющуюся модель, полученную в первом спринте, либо обучите новую модель по своему усмотрению.


In [ ]:
# Базовый код ругается, что это пайплайн, и не даёт сохранить.
# model = joblib.load('/home/mle-user/mle_projects/mle-mlflow/models/model.pkl')
# experiment_id = '4'
# with mlflow.start_run(run_name='log_model', experiment_id=experiment_id) as run:
#     run_id = run.info.run_id
  
#     model_info = mlflow.catboost.log_model(cb_model=model, artifact_path="models") 

Вероятно, ваша модель является объектом Pipeline из scikit-learn, и вы пытаетесь сохранить её с помощью функций, предназначенных для моделей CatBoost.
Для того чтобы решить эту проблему, вам нужно использовать соответствующие методы для сохранения и загрузки модели в зависимости от её типа. Если ваша модель — это Pipeline, то для её сохранения можно использовать joblib или pickle, как вы уже делали для загрузки модели.
Если вы хотите использовать MLflow для логирования модели, убедитесь, что используете правильный флавор (например, mlflow.sklearn для моделей scikit-learn).
Вот пример, как можно логировать модель scikit-learn с помощью MLflow:

In [ ]:
from mlflow.tracking import MlflowClient
model = joblib.load('/home/mle-user/mle_projects/mle-mlflow/models/model.pkl')
experiment_id = '4'

with mlflow.start_run(run_name='log_model', experiment_id=experiment_id) as run:
    run_id = run.info.run_id
    
    # Логируем модель
    model_info = mlflow.sklearn.log_model(sk_model=model, artifact_path="models")

4.Установите библиотеку scikit-learn в ваше окружение: pip install scikit-learn==1.3.1.




Также для успешного выполнения заданий подготовьте следующие пререквизиты:
- Объект модели. Это та самая обученная модель, которую вы планируете логировать с помощью MLflow. Сохраните её в переменную model.
- Предсказанные моделью значения. Полученные прогнозы модели, включая вероятности положительного исхода и бинарные значения, которые вы определили. - Сохраните эти значения в переменные: prediction — бинарные значения 0 или 1, proba — вероятности.
- Истинные метки и данные для предсказания. Первое — это истинные значения целевой переменной для вашего тестового набора данных. Сохраните их в переменную y_test. Второе — датасет с признаками для модели, который уже сохранён в переменную X_test.

In [ ]:
# Загрузим данные:
# df = pd.read_csv('users_churn.csv') - проблема здесь - нужны сразу предлобработанные данные

# Поделим данные:
X = df.drop('Target',axis=1)
y = df['Target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Получение бинарных значений
prediction = model.predict(X_test)

# Получение вероятностей положительного исхода
proba = model.predict_proba(X_test)